<a href="https://colab.research.google.com/github/fourmodern/DRLHO/blob/master/chapter5/05_Tabular_Learning_and_the_Bellman_Equation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. Tabular Learning and the Bellman Equation


* Value, state, and optimality
* The Bellman equation of optimality
* Value of action
* The value iteration method
* Value iteration in practice
* Q-learning for FrozenLake

In [0]:
##prepare
!pip install tensorboardX > /dev/null 2>&1
!pip install -U tensorflow==2.0.0-beta0 > /dev/null 2>&1


In [17]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Value, state, and optimality

<p align="center"><img src="https://github.com/fourmodern/DRLHO/raw/master/chapter5/img/f1.jpeg" width=800 /></p>
<p align="center"><img src="https://github.com/fourmodern/DRLHO/raw/master/chapter5/img/f2.jpeg" width=800 /></p>

## The Bellman equation of optimality

<p align="center"><img src="https://github.com/fourmodern/DRLHO/raw/master/chapter5/img/f3.jpeg" width=800 /></p>
<p align="center"><img src="https://github.com/fourmodern/DRLHO/raw/master/chapter5/img/f4.jpeg" width=800 /></p>
<p align="center"><img src="https://github.com/fourmodern/DRLHO/raw/master/chapter5/img/f5.jpeg" width=800 /></p>

## Value of action

<p align="center"><img src="https://github.com/fourmodern/DRLHO/raw/master/chapter5/img/f6.jpeg" width=600 /></p>
<p align="center"><img src="https://github.com/fourmodern/DRLHO/raw/master/chapter5/img/f7.jpeg" width=200 /></p>
<p align="center"><img src="https://github.com/fourmodern/DRLHO/raw/master/chapter5/img/f8.jpeg" width=400 /></p>

<p align="center"><img src="https://github.com/fourmodern/DRLHO/raw/master/chapter5/img/f9.jpeg" width=800 /></p>
<p align="center"><img src="https://github.com/fourmodern/DRLHO/raw/master/chapter5/img/f10.jpeg" width=800 /></p>
<p align="center"><img src="https://github.com/fourmodern/DRLHO/raw/master/chapter5/img/f11.jpeg" width=800 /></p>
<p align="center"><img src="https://github.com/fourmodern/DRLHO/raw/master/chapter5/img/f12.jpeg" width=600 /></p>
<p align="center"><img src="https://github.com/fourmodern/DRLHO/raw/master/chapter5/img/f13.jpeg" width=600 /></p>
<p align="center"><img src="https://github.com/fourmodern/DRLHO/raw/master/chapter5/img/f14.jpeg" width=600 /></p>

## The value iteration method

<p align="center"><img src="https://github.com/fourmodern/DRLHO/raw/master/chapter5/img/f16.jpeg" width=800 /></p>

## Value iteration in practice



<p align="center"><img src="https://github.com/fourmodern/DRLHO/raw/master/chapter5/img/frozen.jpeg" width=500 /></p>
<p align="center"><img src="https://github.com/fourmodern/DRLHO/raw/master/chapter5/img/f17.jpeg" width=800 /></p>

In [2]:
import tensorboardcolab as tb
tbc = tb.TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
http://3ad68b93.ngrok.io


In [0]:
import gym
import collections
from tensorboardX import SummaryWriter

In [0]:
ENV_NAME = "FrozenLake-v0"
GAMMA = 0.9
TEST_EPISODES = 20

In [0]:
class Agent:
    def __init__(self):
        self.env = gym.make(ENV_NAME)
        self.state = self.env.reset()
        self.rewards = collections.defaultdict(float)
        self.transits = collections.defaultdict(collections.Counter)
        self.values = collections.defaultdict(float)

    def play_n_random_steps(self, count):
        for _ in range(count):
            action = self.env.action_space.sample()
            new_state, reward, is_done, _ = self.env.step(action)
            self.rewards[(self.state, action, new_state)] = reward
            self.transits[(self.state, action)][new_state] += 1
            self.state = self.env.reset() if is_done else new_state

    def calc_action_value(self, state, action):
        target_counts = self.transits[(state, action)]
        total = sum(target_counts.values())
        action_value = 0.0
        for tgt_state, count in target_counts.items():
            reward = self.rewards[(state, action, tgt_state)]
            action_value += (count / total) * (reward + GAMMA * self.values[tgt_state])
        return action_value

    def select_action(self, state):
        best_action, best_value = None, None
        for action in range(self.env.action_space.n):
            action_value = self.calc_action_value(state, action)
            if best_value is None or best_value < action_value:
                best_value = action_value
                best_action = action
        return best_action

    def play_episode(self, env):
        total_reward = 0.0
        state = env.reset()
        while True:
            action = self.select_action(state)
            new_state, reward, is_done, _ = env.step(action)
            self.rewards[(state, action, new_state)] = reward
            self.transits[(state, action)][new_state] += 1
            total_reward += reward
            if is_done:
                break
            state = new_state
        return total_reward

    def value_iteration(self):
        for state in range(self.env.observation_space.n):
            state_values = [self.calc_action_value(state, action)
                            for action in range(self.env.action_space.n)]
            self.values[state] = max(state_values)

In [0]:
test_env = gym.make(ENV_NAME)
agent = Agent()

In [19]:
writer = SummaryWriter(comment="-v-iteration")

iter_no = 0
best_reward = 0.0

while True:
  
  iter_no += 1

  agent.play_n_random_steps(100)
  agent.value_iteration()

  reward = 0.0
  
  for _ in range(TEST_EPISODES):
    reward += agent.play_episode(test_env)
  
  reward /= TEST_EPISODES
  writer.add_scalar("reward", reward, iter_no)
 
  if reward > best_reward:
    print("Best reward updated %.3f -> %.3f" % (best_reward, reward))
    best_reward = reward
  
  if reward > 0.80:
    print("Solved in %d iterations!" % iter_no)
    break

writer.close()

Best reward updated 0.000 -> 0.700
Best reward updated 0.700 -> 0.750
Best reward updated 0.750 -> 0.800
Best reward updated 0.800 -> 0.850
Solved in 56 iterations!


## Q-learning for FrozenLake

In [0]:
import gym
import collections
from tensorboardX import SummaryWriter


In [0]:
ENV_NAME = "FrozenLake-v0"
GAMMA = 0.9
TEST_EPISODES = 20

In [0]:
class Agent:
    def __init__(self):
        self.env = gym.make(ENV_NAME)
        #self.env = wrap_env(gym.make(ENV_NAME))
        self.state = self.env.reset()
        self.rewards = collections.defaultdict(float)
        self.transits = collections.defaultdict(collections.Counter)
        self.values = collections.defaultdict(float)

    def play_n_random_steps(self, count):
        for _ in range(count):
            action = self.env.action_space.sample()
            new_state, reward, is_done, _ = self.env.step(action)
            self.rewards[(self.state, action, new_state)] = reward
            self.transits[(self.state, action)][new_state] += 1
            self.state = self.env.reset() if is_done else new_state

    def select_action(self, state):
        best_action, best_value = None, None
        for action in range(self.env.action_space.n):
            action_value = self.values[(state, action)]
            if best_value is None or best_value < action_value:
                best_value = action_value
                best_action = action
        return best_action

    def play_episode(self, env):
        total_reward = 0.0
        state = env.reset()
        while True:
            action = self.select_action(state)
            new_state, reward, is_done, _ = env.step(action)
            self.rewards[(state, action, new_state)] = reward
            self.transits[(state, action)][new_state] += 1
            total_reward += reward
            if is_done:
                break
            state = new_state
        return total_reward

    def value_iteration(self):
        for state in range(self.env.observation_space.n):
            for action in range(self.env.action_space.n):
                action_value = 0.0
                target_counts = self.transits[(state, action)]
                total = sum(target_counts.values())
                for tgt_state, count in target_counts.items():
                    reward = self.rewards[(state, action, tgt_state)]
                    best_action = self.select_action(tgt_state)
                    action_value += (count / total) * (reward + GAMMA * self.values[(tgt_state, best_action)])
                self.values[(state, action)] = action_value

In [0]:
test_env = gym.make(ENV_NAME)
agent = Agent()

In [25]:
writer = SummaryWriter(comment="-q-iteration")

iter_no = 0
best_reward = 0.0

while True:
  
  iter_no += 1

  agent.play_n_random_steps(100)
  agent.value_iteration()

  reward = 0.0
  
  for _ in range(TEST_EPISODES):
    reward += agent.play_episode(test_env)
  
  reward /= TEST_EPISODES
  writer.add_scalar("reward", reward, iter_no)
 
  if reward > best_reward:
    print("Best reward updated %.3f -> %.3f" % (best_reward, reward))
    best_reward = reward
  
  if reward > 0.80:
    print("Solved in %d iterations!" % iter_no)
    break

writer.close()

Best reward updated 0.000 -> 0.100
Best reward updated 0.100 -> 0.600
Best reward updated 0.600 -> 0.700
Best reward updated 0.700 -> 0.850
Solved in 36 iterations!
